# Session 1b - Inmas Workshop Machine Learning Workshop, January 13-14, 2024

Instructor: Christian Kuemmerle - kuemmerle@uncc.edu 

## High-Dimensional Geometry, Sparse Regression, Feature Selection

First, we load packages and functions that we will need.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Geometry of High-Dimensional Spaces

First, we explore the phenomenon of concentration of measure in high dimensions through simple examples as a warm-up.

We compare the behavior of multivariate normal distribution in high dimensions with low-dimensional ones.

In [ ]:
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

In [ ]:
# Create a random number generator with a fixed seed for reproducibility
rng = np.random.default_rng(100)

**Randomly generate 1000 points from an $d$-dimensional standard Gaussian distribution and plot the histogram of the distances between those 1000 points and the origin. Do this for $d=1$, $d=2$, $d=100$ and $d=50000$.** <br>

Hint: You can use `numpy.random.randn` to sample standard Gaussians and `numpy.linalg.norm` to compute vector norms. You can look into the `numpy` documentation for more information. 

In [ ]:
### Add your code here ###





What do the histograms tell us?

The high-dimensional normal distribution is close to a spherical-type of distribution, whereas the $2$-dimensional one is rather spread out.

**Randomly generate two sets of 500 points from a $d$-dimensional standard gaussian distribution and plot the histogram of the pairwise distances between the two sets with $d=1$, $d=2$, $d=100$ and $d=50000$.**)

In [ ]:
from scipy.spatial import distance_matrix

In [ ]:
# Create a random number generator with a fixed seed for reproducibility
rng = np.random.default_rng(100)

In [ ]:
d = [1, 2,100,5000]
n = 500
npoints = n**2

In [ ]:
### Add your code here ###







What does the histogram tell us?

The pairwise distances of i.i.d. samples of a high-dimensional Gaussian concentrate very much for large $d$. In particular, this means that there are very few points that are "close" to each other in high dimensions! They seem to have almost all roughly the same distance to each other, which is remarkable.

## The geometry of the high-dimensional sphere

**Randomly generate 1000 points uniformly from a 100-dimensional unit ball and plot the histogram of the distances between those 1000 points and the origin.** <br> <br>
**Hint:** There are several ways how to solve this problem.
   - One of the uses high-dimensional Gaussian distribution, see also: https://stackoverflow.com/questions/5408276/sampling-uniformly-distributed-random-points-inside-a-spherical-volume/
   - Another possibility includes _rejection sampling_: To generate one point, one can uniformly draw from the unit box with radius $1$, and reject samples that are far enough away.

In [ ]:
from scipy.special import gammainc
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

In [ ]:
    # Create a random number generator with a fixed seed for reproducibility
rng = np.random.default_rng(100)

In [ ]:
# define a function for generating npoints points uniformly from a n-dimensional unit ball
# Hint: use high-dimensional gaussian distribution and incomplete gamma function to generate the points, see https://stackoverflow.com/questions/5408276/sampling-uniformly-distributed-random-points-inside-a-spherical-volume/ for details
def sample_ball(npoints,n):
    p = 
    return p

In [ ]:
p = sample_ball(1000,100)

In [ ]:
n_bins = 20
fig3, ax3 = plt.subplots(1, sharey=True, tight_layout=True)
# We can set the number of bins with the *bins* keyword argument.
ax3.hist(np.linalg.norm(p, axis=1), bins=n_bins)
plt.show()

**What does the histogram tell us?**


# Sparse Regression

We return to the [Boston housing dataset](https://www.openml.org/d/531) and apply ideas from sparse regularization in order to obtain more interpretable models (feature selection).

### Lasso regression with cross-validated regularization  paramter (hyperparameter optimization via grid search)

We will apply the described procedure for [Lasso regression](https://web.stanford.edu/~hastie/StatLearnSparsity/) ("Least absolute shrinkage and selection operator"), which promotes _sparsity_ in the coefficients of the linear model. As the learned model tends to have a sparse coefficient vector, it leads to a better _interpretability_ of the model as we can identify which features are used in the model, and which are not.

First, we load again the data, and put it into a `pandas.DataFrame` data structure.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
# Load data set
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO']
df_boston = pd.DataFrame(data[:,0:11],columns = column_names)
df_boston.assign(MEDV=target)

We paste some of the functions and commands we used in the notebook `session1a_RidgeCrossVal.ipynb` to preprocess the data.

In [ ]:
X = df_boston[column_names]
y = target
print(X.shape,y.shape)

**Quick question: Do you see why the numpy arrays `X` and `y` have these dimensions?**

In [ ]:
# we continue with the preprocessing
scaler = StandardScaler()
X_scaled =scaler.fit_transform(X)
test_share = 0.2

X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,random_state=10,test_size=test_share)

We paste some of the functions we used in part 1. In particular, these functions are meant have been used to
   - plot correlation matrices,
   - evaluate the $R^2$ value on training and test or validation set for a given predictive model,
   - plot errors.

In [ ]:
def plot_correlation_true_predicted(y_train,y_test,model,figsize=(8,8)): # Plot correlation matrix of expected vs. "measured" median housing price for model
    plt.figure(figsize=figsize)
    plt.scatter(y_train,model.predicted_train)
    plt.scatter(y_test, model.predicted_test)
    maxx= max(np.max(model.predicted_train),np.max(model.predicted_test),np.max(y_train),np.max(y_test))
    ax=plt.gca()
    ax.set_ylim(0, 1.02*maxx)
    ax.set_xlim(0, 1.02*maxx)
    ax.legend(["training data","test data"], loc=0)
    ax.plot(ax.get_xlim(), ax.get_ylim(), ls="--", c=".3")
    ax.set(ylabel='predicted median housing price (in $1000)', xlabel='true median housing price (in $1000)')
    ax.set(title="Correlation between true and predicted prices for model "+str(model))

In [ ]:
def eval_prediction_train_test(model,X_train,X_test,y_train,y_test,plot=True):
    model.score_train = model.score(X_train, y_train)
    model.score_test = model.score(X_test, y_test)
    model.predicted_test = model.predict(X_test)
    model.predicted_train = model.predict(X_train)
    print("R^2 value of model",str(model),"on the train set: %f" % model.score_train)
    print("R^2 value of model",str(model),"on the test set: %f" % model.score_test)
    if plot:
        plot_correlation_true_predicted(y_train,y_test,model)
    return model

In [ ]:
# Define plotting function for training und test errors for different model complexities:
def plot_training_test(alphas,train_error,test_error):
    plt.figure()
    plt.plot(alphas,train_error)
    plt.plot(alphas,test_error)
    ax = plt.gca()
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set(xlabel='alpha', ylabel='mean squared error')
    ax.legend(["training data","test data"], loc=0)

Furthermore, the next function wraps `GridSearchCV` for $k$-fold crossvalidation (with $k$=`cv`). See below for how it can be used.

In [ ]:
def gridsearch_crossvalidation_model(model,X,y,parameters,cv=5,plot=True):
    gridsearch = GridSearchCV(model,param_grid=parameters,scoring='r2',return_train_score=True,cv=cv) # cv
    gridsearch.fit(X, y)
    gridsearch.train_errors = gridsearch.cv_results_['mean_train_score']
    gridsearch.test_errors  = gridsearch.cv_results_['mean_test_score']
    if plot: # Plot training und test errors for different model complexities of ridge regression
        plot_training_test(model,gridsearch.train_errors,gridsearch.test_errors)
    return gridsearch

### Lasso and Cross Validation

Repeat previous experiments from `session1a_RidgeCrossVal.ipynb` about accuracy evaluation and 5-fold cross validation with Lasso instead of ridge regression. <br> 

First, we train Lasso for fixed $\alpha$.

In [ ]:
from sklearn.linear_model import Lasso
alpha_lasso = 1
lasso = # train LASSO for fixed regularization parameter alpha=1
lasso = eval_prediction_train_test(lasso,X_train,X_test,y_train,y_test)
print("Number of features used for model "+str(lasso)+": {}".format(np.sum(lasso.coef_ != 0)))

Now, **run cross validation to optimize regularization parameter alpha of Lasso on the training set**.

In [ ]:
## td logarithmically interpolated values between 10^(-5) and 10^(9)
parameters = {'alpha':alphas}
# Run cross validation to optimize regularization parameter alpha of Lasso
gridsearch_lasso = 
lasso_optimized = 
print("Number of features used for model "+str(lasso_optimized)+": {}".format(np.sum(lasso_optimized.coef_ != 0)))

We ran Lasso first for $\alpha=1$, which leads to a model with fewer non-zero coefficients. However, we see that the resulting errors are likely larger, i.e., the $R^2$ values for test and training set are smaller than for ridge regression.
 
Optimizing $\alpha$ by cross validation leads to a model with more non-zero coefficeints, but better predictive properties (also improving on ridge regression in this case).

In [ ]:
# %% Summarize results
from sklearn.metrics import mean_squared_error
results = {'Lasso': [lasso_optimized.score_test,mean_squared_error(lasso_optimized.predict(X_test), y_test)]}
df_results = pd.DataFrame(data=results,index=['R2 on test set','Test MSE']) 
print(df_results)

To explore the interpretability of our models, we plot the regression coefficients.

In [ ]:
gridsearch_ridge = gridsearch_crossvalidation_model(Ridge(),X_train,y_train,parameters,plot=False)
ridge_optimized = gridsearch_ridge.best_estimator_

We plot coefficients for the two different models - Lasso and Ridge regression.

In [ ]:
plt.figure(figsize=(14,14))
plt.plot(lasso_optimized.coef_, '^', label=str(lasso_optimized))
plt.plot(ridge_optimized.coef_, 'v', label=str(ridge_optimized))
plt.legend(ncol=2, loc=(0, 1.05))
plt.xlabel("Coefficient index")
plt.ylabel("Coefficient magnitude")
plt.xticks(ticks=np.arange(0,11),labels=df_boston.columns[0:11])
ax = plt.gca()

So far, we have used only the $11$ original features to run all models (Ridge/Lasso). In the first notebook `session1a_RidgeCrossVal.ipynb`, we saw that enriching the features by preprocessing, e.g., through a polynomial map can improve the performance of linear models significantly.

**In the next task, we apply the above models again to the the a set of predictor variables consisting of polynomials of degree of at most three of the original features.**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Create regressor variables with polynomial features
degree_polynomial = 3
poly = 

X_train_poly = 
X_test_poly = 

# Rescale variables
scaler = 
X_train_poly = 
X_test_poly = 

#X_poly      = poly.transform(X_scaled)
print("X_train.shape: {}".format(X_train.shape)) 
print("X_train_poly.shape: {}".format(X_train_poly.shape))

We first run ridge regression for fixed alpha regularization.

In [ ]:
alpha_ridge = 1
ridge = 

ridge = eval_prediction_train_test(ridge,X_train_poly,X_test_poly,y_train,y_test)

Run ridge regression for cross validated regularization parameter alpha.

In [ ]:
gridsearch_ridge = gridsearch_crossvalidation_model(Ridge(),X_train_poly,y_train,parameters,plot=False)
ridge_optimized = gridsearch_ridge.best_estimator_
ridge_optimized = eval_prediction_train_test(ridge_optimized,X_train_poly,X_test_poly,y_train,y_test)

**Note:** In the following cell, convergence warnings might arise. This is due to the fact that we do not have simple closed-from solution here, but need to run an iterative algorithm to minimize the Lasso objective. The optimizer is based on coordinate descent and run by `scikit-learn` in the backend. Potentially, a different algorithm optimization the same objective might here be useful.|

In [ ]:
# Run lasso for cross validated regularization parameter alpha
gridsearch_lasso = gridsearch_crossvalidation_model(Lasso(max_iter=1000,warm_start=True),X_train_poly,y_train,parameters,plot=False)
lasso_optimized = gridsearch_lasso.best_estimator_
lasso_optimized = eval_prediction_train_test(lasso_optimized,X_train_poly,X_test_poly,y_train,y_test)
print("Number of features used for model "+str(lasso_optimized)+": {}".format(np.sum(lasso_optimized.coef_ != 0)))

For Lasso regression with cross-validated regularization parameter $\alpha$, we can now look at the _sparsity_, i.e., the number of non-zero coefficients in the coefficient vectors:

In [ ]:
# vars(lasso_optimized)
lasso_optimized.coef_

We see that the this coefficient vector has a lot of zero entries and just few-nonzero ones. In particular, while the dimensionality of the vector is

In [ ]:
np.shape(lasso_optimized.coef_)

the number of non-zeros is

In [ ]:
np.count_nonzero(lasso_optimized.coef_)

Summarizing the performance of the different regression methods using polynomial features, we obtain:

In [ ]:
# Summarize results for models trained on new features
from sklearn.metrics import mean_squared_error
results_poly = {'Ridge': [ridge_optimized.score_test, mean_squared_error(ridge_optimized.predict(X_test_poly), y_test)], 
           'Lasso': [lasso_optimized.score_test,mean_squared_error(lasso_optimized.predict(X_test_poly), y_test)]}
df_results_poly = pd.DataFrame(data=results_poly,index=['R2 on test set','Test MSE']) 
print(df_results_poly)

Comparing it with the $R^2$ values above, we see that working with polynomial features clearly improved the performance!

Finally, we create a plot that visualizes the differences between the coefficients returned by the methods.

In [ ]:
# %% Plot coefficients for different models
plt.figure(figsize=(14,14))
plt.plot(lasso_optimized.coef_, '^', label=str(lasso_optimized))
plt.plot(ridge_optimized.coef_, 'v', label=str(ridge_optimized))
plt.legend(ncol=2, loc=(0, 1.05))
#plt.ylim(-25, 25)
plt.xlabel("Coefficient index")
plt.ylabel("Coefficient magnitude")
ax = plt.gca()
#ax.set_aspect(1)

We observe that the coefficient vector returned by ridge regression has many small, but non-zero coefficients, whereas Lasso has few non-zero, but larger coefficients.
 
 
Relating the coefficient index numbers with the original polynomial features created, we the see therefore that the sparse regression method (Lasso) given us further options to _interpret_ model:
 
Each coefficient index corresponds to a certain monomial created from the 11 variables

In [ ]:
df_boston.columns

so that we can learn which (combinations of) features are able to create a good predictive model. The fewer non-zero coefficients we have, the "simpler" and "easier to interpret" the model becomes.

**Question: Can you identify which variables / variable combinations are the most significant in the Lasso model trained above for polynomial features?**

In [ ]:
### Add your code here ###